# Tech - profiling

Le [profilage de code](https://fr.wikipedia.org/wiki/Profilage_de_code) ou *profiling* en anglais signifie qu'on mesure le temps passé dans chaque partie d'un programme pour en découvrir les parties les plus coûteuses et les améliorer. On souhaite toujours accélérer un programme trop lent, le profiling permet de savoir sur quelles parties se concentrer.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

Autre lecture : [Exemple de profiling](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/profiling_example.html#profilingexamplerst).

## Enoncé

On applique le profiling au code suivant pour découvrir laquelle de ces fonctions est la plus lente. Vous pouvez changer ce code pour un de ceux que vous avez écrits précédemment.

In [2]:
def _gini_sort(Y):
    return list(sorted(Y))


def _gini_init(Y):
    return [[1, y] for y in Y]


def _gini_cumsum(couples):
    for i in range(1, len(couples)):
        couples[i][0] += couples[i-1][0]
        couples[i][1] += couples[i-1][1]
    for i in range(0, len(couples)):
        couples[i][0] /= couples[-1][0]
        couples[i][1] /= couples[-1][1]
    return couples

    
def _gini_final(couples):
    g = couples[0][0] * couples[0][1]
    n = len(couples)

    for i in range(1, n):
        dx = couples[i][0] - couples[i-1][0]
        y = couples[i-1][1] + couples[i][1]
        g += dx * y

    return 1. - g / 2


def gini(Y):
    Y = _gini_sort(Y)
    couples = _gini_init(Y)
    couples = _gini_cumsum(couples)
    return _gini_final(couples)


gini([1, 1, 1, 1, 1]), gini([0, 0, 0, 0, 100000])

(0.5, 0.9)

### Exercice 1 : profiler le code précédent

Le langage python dispose d'un module qui mesure le temps passé dans chaque fonction [profile](https://docs.python.org/3/library/profile.html). Il faut l'utiliser.

### Exercice 2 : changer la fonction _gini_final si possible en plus rapide



### Exercice 3 : vous améliorez la fonction _gini_final, profilez pour savoir de combien ?

### Exercice 4 : utiliser d'autres modules de profiling

Si c'est possible, comme [pyinstrument](https://github.com/joerick/pyinstrument) ou [py-spy](https://github.com/benfred/py-spy). Il y a deux façons de faire du profiling :

* **déterministe :** on mesure le temps passé dans chaque fonction, à chaque appel, ce type de profilage n'est souvent possible que pour des langages interprétés,
* **statistique :** tous les centièmes de secondes, on regarde quelle ligne de quelle fonction le programme exécute. On compte ensuite combien de fois la ligne exécutée était dans une fonction pour déterminer le temps passé dans chaque fonction.

Les deux modules proposés sont statistiques, le premier est déterministe. Bien évidemment, le fait de mesurer le temps passé prend du temps également, plus on s'arrête souvant, plus l'exécution est ralentie.

### Exercice 5 : la fonction _gini_cumsum contient deux boucles. Quelle est la plus rapide ?

## Réponses

### Exercice 1 : profiler le code précédent

On reprend un des exemples de cette page [The Python Profilers](https://docs.python.org/3/library/profile.html#module-cProfile).

In [8]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()

gini([1, 1, 1, 1, 1])

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         57 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 c:\python372_x64\lib\site-packages\IPython\core\interactiveshell.py:3293(run_code)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 c:\python372_x64\lib\codeop.py:132(__call__)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-405f7b24fbdc>:6(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-2-3b0b0c672aa1>:31(gini)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        1    0.000    0.000    0.000    0.000 <ipython-input-2-3b0b0c672aa1>:9(_gini_cumsum)
        1    0.000    0.000    0.000    0.000 <ipython-input-2-3b0b0c672aa1>:19(_gini_final)
        2    0.000    0.000    0.000    0.000 c:\python372_x64\lib\contextlib.py:107(__enter__)
        2    0.

Tous les temps sont nuls. Le code est trop rapide. Il faut soit exécuter le code plusieurs fois, soit prendre un tableau plus grand.

In [9]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()

for i in range(1000):
    gini([1 for i in range(1000)])

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         11047 function calls in 1.274 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.274    0.637 c:\python372_x64\lib\site-packages\IPython\core\interactiveshell.py:3293(run_code)
        2    0.000    0.000    1.274    0.637 {built-in method builtins.exec}
        1    0.043    0.043    1.274    1.274 <ipython-input-5-b1075e275d99>:6(<module>)
     1000    0.002    0.000    1.190    0.001 <ipython-input-2-3b0b0c672aa1>:31(gini)
     1000    0.673    0.001    0.673    0.001 <ipython-input-2-3b0b0c672aa1>:9(_gini_cumsum)
     1000    0.360    0.000    0.361    0.000 <ipython-input-2-3b0b0c672aa1>:19(_gini_final)
     1000    0.001    0.000    0.142    0.000 <ipython-input-2-3b0b0c672aa1>:5(_gini_init)
     1000    0.141    0.000    0.141    0.000 <ipython-input-2-3b0b0c672aa1>:6(<listcomp>)
     1000    0.041    0.000    0.041    0.000 <ipython-input-5-b1075e275d99>:7(<listcomp>)
     10

### Exercice 2 : changer la fonction _gini_final si possible en plus rapide

In [10]:
def _gini_final_faster(couples):
    g = couples[0][0] * couples[0][1]
    n = len(couples)

    # par construction, tous les dx sont identiques, cela fait des calculs en moins
    # le code suivant est plus rapide mais peut encore être optimisé
    dx = couples[0][0]
    sy = couples[0][0]
    for i in range(1, n):        
        sy = couples[i-1][1] + couples[i][1]
        g += dx * sy

    return 1. - g / 2


def gini_faster(Y):
    Y = _gini_sort(Y)
    couples = _gini_init(Y)
    couples = _gini_cumsum(couples)
    return _gini_final_faster(couples)


(gini_faster([0, 0, 0, 0, 5000]), gini_faster([1, 1, 1, 1, 1]),
 gini([0, 0, 0, 0, 5000]), gini([1, 1, 1, 1, 1]))

(0.9, 0.5, 0.9, 0.5)

### Exercice 3 : vous améliorez la fonction _gini_final, profilez pour savoir de combien ?

On appelle les deux fonctions pour pouvoir comparer.

In [11]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()

for i in range(1000):
    gini([1 for i in range(1000)])
    gini_faster([1 for i in range(1000)])

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         22047 function calls in 2.427 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.427    1.213 c:\python372_x64\lib\site-packages\IPython\core\interactiveshell.py:3293(run_code)
        2    0.000    0.000    2.427    1.213 {built-in method builtins.exec}
        1    0.082    0.082    2.427    2.427 <ipython-input-47-48004205d1de>:6(<module>)
     2000    1.344    0.001    1.345    0.001 <ipython-input-37-6d8b3d4c048c>:9(_gini_cumsum)
     1000    0.002    0.000    1.193    0.001 <ipython-input-37-6d8b3d4c048c>:31(gini)
     1000    0.002    0.000    1.068    0.001 <ipython-input-46-25d64f007e27>:15(gini_faster)
     1000    0.358    0.000    0.358    0.000 <ipython-input-37-6d8b3d4c048c>:19(_gini_final)
     2000    0.002    0.000    0.314    0.000 <ipython-input-37-6d8b3d4c048c>:5(_gini_init)
     2000    0.312    0.000    0.312    0.000 <ipython-input-37-6d8b3d4c048c>:6(<listcomp>

La seconde version est plus rapide.

### Exercice 4 : utiliser d'autres modules de profiling

[pyinstrument](https://github.com/joerick/pyinstrument) n'est pas peut-être pas installé. Il fonctionne de la même façon.

In [12]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

for i in range(1000):
    gini([1 for i in range(1000)])
    gini_faster([1 for i in range(1000)])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 23:55:20  Samples:  2658
 /_//_/// /_\ / //_// / //_'/ //     Duration: 3.506     CPU time: 3.328
/   _/                      v3.1.3

Program: c:\python372_x64\lib\site-packages\ipykernel_launcher.py -f C:\Users\xavie\AppData\Roaming\jupyter\runtime\kernel-86977e98-0575-4ce5-83b5-3c2d9e6b311c.json

3.506 run_ast_nodes  IPython\core\interactiveshell.py:3142
└─ 3.505 run_code  IPython\core\interactiveshell.py:3293
   └─ 3.505 <module>  <ipython-input-48-23d6f1dc379d>:6
      ├─ 1.830 gini  <ipython-input-37-6d8b3d4c048c>:31
      │  ├─ 0.955 _gini_cumsum  <ipython-input-37-6d8b3d4c048c>:9
      │  ├─ 0.606 _gini_final  <ipython-input-37-6d8b3d4c048c>:19
      │  └─ 0.237 _gini_init  <ipython-input-37-6d8b3d4c048c>:5
      │     └─ 0.237 <listcomp>  <ipython-input-37-6d8b3d4c048c>:6
      ├─ 1.456 gini_faster  <ipython-input-46-25d64f007e27>:15
      │  ├─ 0.994 _gini_cumsum  <ipython-input-37-6d8b3d4c048c>:9
      │  ├─ 0.274 _gini_final_fa

C'est plus facile à lire.

### Exercice 5 : la fonction _gini_cumsum contient deux boucles. Quelle est la plus rapide ?

Pour ce faire, on découpe la fonction en deux.

In [13]:
def _gini_cumsum2a(couples):
    for i in range(1, len(couples)):
        couples[i][0] += couples[i-1][0]
        couples[i][1] += couples[i-1][1]
    return couples


def _gini_cumsum2b(couples):
    for i in range(0, len(couples)):
        couples[i][0] /= couples[-1][0]
        couples[i][1] /= couples[-1][1]
    return couples


def _gini_cumsum2(couples):
    _gini_cumsum2a(couples)
    _gini_cumsum2b(couples)
    return couples


def gini2(Y):
    Y = _gini_sort(Y)
    couples = _gini_init(Y)
    couples = _gini_cumsum2(couples)
    return _gini_final(couples)


gini2([1, 1, 1, 1, 1]), gini2([0, 0, 0, 0, 100000])

(0.5, 0.9)

In [14]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()

for i in range(1000):
    gini2([1 for i in range(1000)])
    gini_faster([1 for i in range(1000)])

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         24047 function calls in 2.505 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.505    1.252 c:\python372_x64\lib\site-packages\IPython\core\interactiveshell.py:3293(run_code)
        2    0.000    0.000    2.505    1.252 {built-in method builtins.exec}
        1    0.092    0.092    2.505    2.505 <ipython-input-50-997a7e374f11>:6(<module>)
     1000    0.002    0.000    1.221    0.001 <ipython-input-49-2fd5204eb0f7>:21(gini2)
     1000    0.002    0.000    1.103    0.001 <ipython-input-46-25d64f007e27>:15(gini_faster)
     1000    0.695    0.001    0.696    0.001 <ipython-input-37-6d8b3d4c048c>:9(_gini_cumsum)
     1000    0.001    0.000    0.684    0.001 <ipython-input-49-2fd5204eb0f7>:15(_gini_cumsum2)
     1000    0.368    0.000    0.368    0.000 <ipython-input-37-6d8b3d4c048c>:19(_gini_final)
     1000    0.349    0.000    0.349    0.000 <ipython-input-49-2fd5204eb0f7>:1(_gini

Elles sont aussi rapides l'une que l'autre. On peut néanmoins en accélérer une.

In [15]:
def _gini_cumsum3b(couples):
    total0 = couples[-1][0]
    total1 = couples[-1][1]
    for i in range(0, len(couples)):
        couples[i][0] /= total0
        couples[i][1] /= total1
    return couples


def _gini_cumsum3(couples):
    _gini_cumsum2a(couples)
    _gini_cumsum3b(couples)
    return couples


def gini3(Y):
    Y = _gini_sort(Y)
    couples = _gini_init(Y)
    couples = _gini_cumsum3(couples)
    return _gini_final(couples)


gini2([1, 1, 1, 1, 1]), gini2([0, 0, 0, 0, 100000])

(0.5, 0.9)

In [16]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()

for i in range(1000):
    gini2([1 for i in range(1000)])
    gini3([1 for i in range(1000)])

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         26047 function calls in 2.702 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.702    1.351 c:\python372_x64\lib\site-packages\IPython\core\interactiveshell.py:3293(run_code)
        2    0.000    0.000    2.702    1.351 {built-in method builtins.exec}
        1    0.096    0.096    2.702    2.702 <ipython-input-52-5ce2791feae5>:6(<module>)
     1000    0.002    0.000    1.316    0.001 <ipython-input-49-2fd5204eb0f7>:21(gini2)
     1000    0.002    0.000    1.200    0.001 <ipython-input-51-492a2c865fd9>:16(gini3)
     2000    0.775    0.000    0.776    0.000 <ipython-input-37-6d8b3d4c048c>:19(_gini_final)
     2000    0.745    0.000    0.745    0.000 <ipython-input-49-2fd5204eb0f7>:1(_gini_cumsum2a)
     1000    0.001    0.000    0.742    0.001 <ipython-input-49-2fd5204eb0f7>:15(_gini_cumsum2)
     1000    0.001    0.000    0.636    0.001 <ipython-input-51-492a2c865fd9>:10(_gini_cu

C'est mieux même si l'amélioration ne paraît pas nécessairement significative par rapport au temps total, le résultat l'est si on regarde le temps fonction par fonction.